In [ ]:
# !!! testing purpose !!!
# this cell is for testing the translation component 
# try uncomment the 2 following line to see the fr translation of the module
#import os
#os.environ["CUSTOM_LANGUAGE"] = "fr"

In [ ]:
from sepal_ui import sepalwidgets as sw
from component.message import cm

In [ ]:
# Create an appBar
app_bar = sw.AppBar(cm.app.title)

In [ ]:
# Create a footer
# the footer will be display in the bottom of the page 
# it's not a mandatory feature 
app_footer = sw.Footer(cm.app.footer.format(2020))

In [ ]:
# load the patial files
%run 'aoi_ui.ipynb'
%run 'default_process_ui.ipynb'
%run 'about_ui.ipynb'

# Gather all the partial tiles that you created previously
app_content = [
    aoi_tile, 
    default_process_tile, 
    default_result_tile, 
    about_tile, 
    disclaimer_tile
]

In [ ]:
# create a drawer for each group of tile 
# use the DrawerItem widget from sepalwidget (name_of_drawer, icon, the id of the widgets you want to display)
# use the display_tile() method to link the times with the drawer items
items = [
    sw.DrawerItem(cm.app.drawer_item.aoi,             'mdi-map-marker-check', card="aoi_widget").display_tile(app_content),
    sw.DrawerItem(cm.app.drawer_item.default_process, 'mdi-cogs',             card="default_process_tile").display_tile(app_content),
    sw.DrawerItem(cm.app.drawer_item.default_result,  'mdi-chart-bar',        card="default_result_tile").display_tile(app_content),
    sw.DrawerItem(cm.app.drawer_item.about,           'mdi-help-circle',      card="about_widget").display_tile(app_content)
]

# !!! not mandatory !!! 
# Add the links to the code, wiki and issue tracker of your 
code_link = 'https://github.com/ingdanielguerrero/sepal_mgci'
wiki_link = 'https://github.com/ingdanielguerrero/sepal_mgci/blob/master/doc/fake_doc.md'
issue_link = 'https://github.com/ingdanielguerrero/sepal_mgci/issues/new'

# Create the side drawer with all its components 
# The display_drawer() method link the drawer with the app bar
app_drawer = sw.NavDrawer(
    items = items, 
    code  = code_link, 
    wiki  = wiki_link, 
    issue = issue_link
).display_drawer(app_bar.toggle_button)


In [ ]:
# build the Html final app by gathering everything 
app = sw.App(
    tiles    = app_content, 
    appBar   = app_bar, 
    footer   = app_footer, 
    navDrawer= app_drawer
).show_tile('aoi_widget') # id of the tile you want to display

In [ ]:
# display the app
# this final cell will be the only one displaying something in this notebook 
# if you run all this notebook you may see elements displayed on the left side of your screen but it won't work
# it can only be launched with voila as it's creating a full page javascript interface
app

In [4]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future


In [5]:
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider


IntSlider(value=0)

In [6]:
out


Output(outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 'did work 0\n'},))